In [1]:
!pip --quiet install lucid

import tensorflow as tf
from tensorflow.python.framework.graph_util import convert_variables_to_constants
import numpy as np
import scipy
import sys
import lucid.optvis.param as param
import lucid.optvis.render as render
from lucid.misc.io.showing import _image_url, _display_html
from lucid.modelzoo.vision_base import Model

import re

You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
def save_frozen_graph(filename, output_channels):
    output_graph_def = tf.graph_util.convert_variables_to_constants(sess
                                                                    , tf.get_default_graph().as_graph_def()
                                                                    , output_channels)
    with open(filename, "wb") as f:
        f.write(output_graph_def.SerializeToString())

# def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
#     """
#     Freezes the state of a session into a pruned computation graph.

#     Creates a new computation graph where variable nodes are replaced by
#     constants taking their current value in the session. The new graph will be
#     pruned so subgraphs that are not necessary to compute the requested
#     outputs are removed.
#     @param session The TensorFlow session to be frozen.
#     @param keep_var_names A list of variable names that should not be frozen,
#                           or None to freeze all the variables in the graph.
#     @param output_names Names of the relevant graph outputs.
#     @param clear_devices Remove the device directives from the graph for better portability.
#     @return The frozen graph definition.
#     """
#     graph = session.graph
#     with graph.as_default():
#         freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
#         output_names = output_names or []
#         output_names += [v.op.name for v in tf.global_variables()]
#         input_graph_def = graph.as_graph_def()
#         if clear_devices:
#             for node in input_graph_def.node:
#                 node.device = ""
       
#         frozen_graph = tf.compat.v1.graph_util.convert_variables_to_constants(session, input_graph_def,
#                                                       output_names, freeze_var_names)
#         return frozen_graph

In [ ]:
# function to output tensorboard
def tensorboard_view(session):
    log_dir = "/notebooks/yu_gpu_cpu_profile/LeNetVisu/logs/"
    
    writer = tf.summary.FileWriter(log_dir)
    writer.add_graph(sess.graph)
    writer.flush()
    writer.close()
    
    !tensorboard --logdir="/notebooks/yu_gpu_cpu_profile/LeNetVisu/logs/"


In [3]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

img_rows = x_train.shape[1]
img_cols = x_train.shape[2]

#preprocessing data
x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

tf.reset_default_graph()

sess = tf.Session()
sess.run(tf.global_variables_initializer())
  
saver = tf.train.import_meta_graph("LeNet5.meta")
saver.restore(sess, "LeNet5")

ops = sess.graph.get_operations()
tensors_names = [m.name for m in ops]
# we hope that the first name is always the first tensor..
tensor_input = tensors_names[0]
# for the outputs, we check for all the operations with activations..
tensor_outputs = []
for s in tensors_names:
    if re.match(".*?(/Relu|/Softmax)$", s):
        print(s)
        tensor_outputs += [s]

# https://gist.github.com/BadrYoubiIdrissi/b2631fddeabf0d8563c6d51cd2055c56
save_frozen_graph('LeNet5.pb', tensor_outputs)

# creating the frozen graph as in
# https://colab.research.google.com/drive/1PPzeZi5sBN2YRlBmKsdvZPbfYtZI-pHl#scrollTo=eju-KLeJ-kBK
# and
# https://colab.research.google.com/drive/1OFesr3ceAaPGmU_gNtDy1djn_9GZgI81#scrollTo=Za1OnQIyDbpJ&forceEdit=true&offline=true&sandboxMode=true

# frozen_graph = freeze_session(sess, output_names=tensor_outputs)
# tf.train.write_graph(frozen_graph, ".", "LeNet5.pb", as_text=False)

Instructions for updating:
Use standard file APIs to check for files with this prefix.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from LeNet5


INFO:tensorflow:Restoring parameters from LeNet5


conv2d/Relu
conv2d_1/Relu
dense/Relu
dense_1/Softmax
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants


...

Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph


INFO:tensorflow:Froze 8 variables.


INFO:tensorflow:Froze 8 variables.


INFO:tensorflow:Converted 8 variables to const ops.


INFO:tensorflow:Converted 8 variables to const ops.


In [6]:
def show_images(images):
    html = ""
    for image in images:
        data_url = _image_url(image)
        html += '<img width=\"100\" style=\"margin: 10px\" src=\"' + data_url + '\">'
        _display_html(html)

def render_vis(model, objective_f, param_f, steps=512):
    T = render.make_vis_T(model, objective_f, param_f, optimizer=None, transforms=[])
    loss, vis_op, t_image = T("loss"), T("vis_op"), T("input")
    sess = tf.Session()
    sess.as_default()
    sess.run(tf.global_variables_initializer())
    for step in range(steps):
        sess.run(vis_op)
    return sess.run(t_image)

class LeNet5Model(Model):
    model_path = './LeNet5.pb'
    image_shape = [1, 28, 28, 1]
    image_value_range = (0., 1.)
    input_name = tensor_input

model = LeNet5Model()
model.load_graphdef()

# for node in model.graph_def.node:
#     print(node.)

softmax_images = []
print(tensor_outputs[-1], end="")
for softmax_idx in range(10):
    print("{}...".format(softmax_idx), end="")
    with tf.Graph().as_default():
#         _ = render.render_vis(model, tensor_outputs[-1] + ":{}".format(softmax_idx),
#                    param_f=param.color.to_valid_rgb(param.spatial.naive((1,28,28,1))))
        image = render_vis(model, tensor_outputs[-1] + ":{}".format(softmax_idx),
                   steps=128, param_f=param.color.to_valid_rgb(param.spatial.naive((1,28,28,1))))
        logit_images.append(image)
show_images(logit_images)

dense_1/Softmax0...

ValueError: Cannot execute operation using `run()`: No default session is registered. Use `with sess.as_default():` or pass an explicit session to `run(session=sess)`

In [ ]:
import lucid.modelzoo.vision_models as models
from lucid.misc.io import show
import lucid.misc.io.showing as showing
from lucid.misc.channel_reducer import ChannelReducer
import lucid.optvis.param as param
import lucid.optvis.objectives as objectives
import lucid.optvis.render as render
from lucid.misc.io import show, load
from lucid.misc.io.reading import read
from lucid.misc.io.showing import _image_url
import lucid.scratch.web.svelte as lucid_svelte
from lucid.misc.gradient_override import gradient_override_map

def raw_class_group_attr(img, layer, label, group_vecs, override=None): 
    """How much did spatial positions at a given layer effect a output class?"""
    # Set up a graph for doing attribution...
    with tf.Graph().as_default(), tf.Session(), gradient_override_map(override or {}):
        t_input = tf.placeholder_with_default(img, [None, None, 3])
        T = render.import_model(model, t_input, t_input)

        # Compute activations
        acts = T(layer).eval()

        if label is None: return np.zeros(acts.shape[1:-1])

        # Compute gradient
        score = T("softmax2_pre_activation")[0, labels.index(label)]
        t_grad = tf.gradients([score], [T(layer)])[0]   
        grad = t_grad.eval({T(layer) : acts})

        # Linear approximation of effect of spatial position
        return [np.sum(group_vec * grad) for group_vec in group_vecs]
    
def neuron_groups(img, layer, n_groups=6, attr_classes=[]):

    # Compute activations

    with tf.Graph().as_default(), tf.Session():
        t_input = tf.placeholder_with_default(img, [1, 28, 28])
        T = render.import_model(model, t_input, t_input)
        acts = T(layer).eval()

    # We'll use ChannelReducer (a wrapper around scikit learn's factorization tools)
    # to apply Non-Negative Matrix factorization (NMF).

    nmf = ChannelReducer(n_groups, "NMF")
    spatial_factors = nmf.fit_transform(acts)[0].transpose(2, 0, 1).astype("float32")
    channel_factors = nmf._reducer.components_.astype("float32")

    # Let's organize the channels based on their horizontal position in the image

    x_peak = np.argmax(spatial_factors.max(1), 1)
    ns_sorted = np.argsort(x_peak)
    spatial_factors = spatial_factors[ns_sorted]
    channel_factors = channel_factors[ns_sorted]

    # And create a feature visualziation of each group

    param_f = lambda: param.image(80, batch=n_groups)
    obj = sum(objectives.direction(layer, channel_factors[i], batch=i)
            for i in range(n_groups))
    group_icons = render.render_vis(model, obj, param_f, verbose=False)[-1]

    # We'd also like to know about attribution

    # First, let's turn each group into a vector over activations
    group_vecs = [spatial_factors[i, ..., None]*channel_factors[i]
                for i in range(n_groups)]

    attrs = np.asarray([raw_class_group_attr(img, layer, attr_class, group_vecs)
                     for attr_class in attr_classes])

    print(attrs)

In [ ]:
model = LeNet5Model()
model.load_graphdef()

neuron_groups(x_test[3,:,:,:], "conv2d/Relu", 5)

In [ ]:
!ls /usr/local/cuda/include/

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline  
plt.imshow(x_test[3,0,:,:], cmap="gray")

In [ ]:
help(param.color.to_valid_rgb)

In [ ]:
render.

In [ ]:
print(tf.train.load_variable("/notebooks/yu_gpu_cpu_profile/LeNetVisu/", "conv2d/kernel"))

In [ ]:
print(tf.train.load_variable("/notebooks/yu_gpu_cpu_profile/LeNetVisu/", "conv2d/kernel_1"))

In [ ]:
# Check if we can do inference
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# trying to predict with the model
with tf.Session() as sess:
    saver = tf.train.import_meta_graph("LeNet5.meta", clear_devices=True)
    saver.restore(sess, "LeNet5")    
    x_tensor = sess.graph.get_tensor_by_name("conv2d_input:0")
    y_tensor = sess.graph.get_tensor_by_name("dense_1/Softmax:0")
    print(sess.run(y_tensor, feed_dict={x_tensor: x_test[0:1,:,:,:]}))

In [ ]:
# list all tensors from a tensorflow graph
ops = sess.graph.get_operations()
# just the tensor names
tensors_names = [m.name for m in ops]

# all the tensor informations
tensors = [m.values() for m in ops]

# More clear way
for operation in ops:
    print("Operation:",operation.name)
    #for k in operation.inputs:
    #    print(operation.name,"Input ",k.name,k.get_shape())
    #for k in operation.outputs:
    #    print(operation.name,"Output ",k.name)
    print("\n")

In [ ]:
# To check all weights from tensorflow session
vars_global = tf.global_variables()

# get their name and value and put them into dictionary
sess.as_default()
model_vars = {}
for var in vars_global:
    try:
        model_vars[var.name] = var.eval()
    except:
        print("For var={}, an exception occurred".format(var.name))

model_vars

In [ ]:
# To check weights from a tensorflow checkpoint
print(tf.train.load_variable("/notebooks/yu_gpu_cpu_profile/LeNetVisu/", "conv2d/kernel"))

In [17]:
!ls /usr/local/lib/python3.5/dist-packages/

Click-7.0.dist-info
IPython
Jinja2-2.10.dist-info
Keras_Applications-1.0.7.dist-info
Keras_Preprocessing-1.0.9.dist-info
Markdown-3.0.1.dist-info
MarkupSafe-1.1.0.dist-info
OpenGL
PIL
Pillow-5.4.1.dist-info
PyOpenGL-3.1.0.dist-info
Pygments-2.3.1.dist-info
Send2Trash-1.5.0.dist-info
Werkzeug-0.14.1.dist-info
__pycache__
absl
absl_py-0.7.0.dist-info
astor
astor-0.7.1.dist-info
backcall
backcall-0.1.0.dist-info
bleach
bleach-3.1.0.dist-info
bz2file-0.98.dist-info
bz2file.py
cachetools
cachetools-3.1.0.dist-info
click
cycler-0.10.0.dist-info
cycler.py
dateutil
decorator-4.3.2.dist-info
decorator.py
defusedxml
defusedxml-0.5.0.dist-info
easy_install.py
entrypoints-0.3.dist-info
entrypoints.py
enum
enum34-1.1.6.dist-info
filelock-3.0.10.dist-info
filelock.py
future
future-0.17.1.dist-info
gast
gast-0.2.2.dist-info
google
grpc
grpcio-1.18.0.dist-info
h5py
h5py-2.9.0.dist-info
ipykernel
ipykernel-5.1.0.dist-info
ipykernel_launcher.py
ipython-7.2.0.dist-info
ipython_genutils
ipython_genutils-0